In [9]:
#Dependencies
import pandas as pd

In [10]:
#read in data
permits_df = pd.read_csv("../data/sidewalk-cafe-permits.csv")
#drop duplicates but there are none
permits_df.drop_duplicates()
permits_df = permits_df.rename(columns={"PERMIT NUMBER": "permit_number", "DOING BUSINESS AS NAME": "business_name", "ISSUED DATE": "issued_date", "EXPIRATION DATE": "expiration_date", "ADDRESS": "address", "LATITUDE": "latitude", "LONGITUDE": "longitude", "LOCATION": "location"})


In [11]:
permits_df.columns

Index(['permit_number', 'ACCOUNT NUMBER', 'SITE NUMBER', 'LEGAL NAME',
       'business_name', 'issued_date', 'expiration_date', 'PAYMENT DATE',
       'address', 'ADDRESS NUMBER START', 'ADDRESS NUMBER', 'STREET DIRECTION',
       'STREET', 'STREET TYPE', 'CITY', 'STATE', 'ZIP CODE', 'WARD',
       'PRECINCT', 'WARD PRECINCT', 'POLICE DISTRICT', 'latitude', 'longitude',
       'location', 'Community Areas', 'Zip Codes', 'Boundaries - ZIP Codes',
       'Census Tracts', 'Wards'],
      dtype='object')

In [16]:
# permits_df.sort_values("ISSUED DATE", ascending=True).head(5)
import_permits_df = permits_df[["permit_number", "business_name", "issued_date", "expiration_date", "address", "latitude", "longitude"]]
import_permits_df.set_index("permit_number", inplace=True)
import_permits_df
import_permits_df.to_csv("past_permits_for_import.csv")

In [17]:
current_permits = pd.read_csv("../data/current_permits.csv")
current_permits.head()
current_permits = current_permits.rename(columns={"PERMIT NUMBER": "permit_number", "DOING BUSINESS AS NAME": "business_name", "ISSUED DATE": "issued_date", "EXPIRATION DATE": "expiration_date", "ADDRESS": "address", "LATITUDE": "latitude", "LONGITUDE": "longitude", "LOCATION": "location"})


In [18]:
import_current_permits_df = current_permits[["permit_number", "business_name", "issued_date", "expiration_date", "address", "latitude", "longitude"]]
import_current_permits_df
# import_current_permits_df.to_csv("current_permits_for_import.csv")

,permit_number,business_name,issued_date,expiration_date,address,latitude,longitude
0,1143796,TURTLES,02/27/2020,05/31/2021,238 W 33RD ST,41.834685,-87.633095
1,1143980,HOTEL EMC2,04/10/2020,05/31/2021,228 E ONTARIO ST,41.893483,-87.621931
2,1144014,PIAZZA BELLA OSTERIA,02/27/2020,05/31/2021,2114 W ROSCOE ST,41.943289,-87.681528
3,1143772,AU CHEVAL,03/04/2020,05/31/2021,800 W RANDOLPH ST,41.884515,-87.647497
4,1144011,SPACE519,05/22/2020,05/31/2021,200 E CHESTNUT ST,41.898530,-87.621777
...,...,...,...,...,...,...,...
1090,1148345,FRANKLIN ROOM,04/09/2021,02/28/2022,675 N FRANKLIN ST,41.894589,-87.635506
1091,1148368,Kraken Sushi & Beyond,04/13/2021,02/28/2022,4022 N WESTERN AVE,41.954759,-87.688635
1092,1148330,AVEC II,04/13/2021,02/28/2022,141 W ERIE ST,41.893900,-87.632732
1093,1148380,FONS EMPANADAS,04/13/2021,02/28/2022,2566 N CLARK ST,41.929403,-87.642899
